# dump-to-db
Dump generated glyphs to MongoDB

## set up connection

In [1]:
import sys

In [2]:
sys.path.append('/usr/lib/python3/dist-packages/')

In [3]:
%cd ..

import os
import sys
import pymongo
import fonts.font_processing as font_processing

/mnt/d/grad23/fonty


In [4]:
sys.path

['/mnt/d/grad23/fonty/notebooks',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/root/.local/share/virtualenvs/fonty-CHIKu2Gx/lib/python3.10/site-packages',
 '/usr/lib/python3/dist-packages/']

In [5]:
cli_remote = pymongo.MongoClient(
    'mongodb+srv://fonty.hquocfa.mongodb.net/?authSource=%24external&authMechanism=MONGODB-X509&retryWrites=true&w=majority',
    tls=True,
    tlsCertificateKeyFile='/mnt/d/grad23/fonty/auth/X509-cert-6658948524510096383.pem'
)

In [6]:
cli_remote

MongoClient(host=['ac-5dsaw84-shard-00-00.hquocfa.mongodb.net:27017', 'ac-5dsaw84-shard-00-01.hquocfa.mongodb.net:27017', 'ac-5dsaw84-shard-00-02.hquocfa.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='$external', authmechanism='MONGODB-X509', retrywrites=True, w='majority', replicaset='atlas-32tpky-shard-0', tls=True, tlscertificatekeyfile='/mnt/d/grad23/fonty/auth/X509-cert-6658948524510096383.pem')

## collect fonts

In [33]:
path = '/mnt/d/grad23/opensans/fonts/ttf'

In [34]:
fonts = [
    os.path.join(path, font_name)
    for font_name
    in os.listdir(path)
]

In [35]:
fonts

['/mnt/d/grad23/opensans/fonts/ttf/OpenSans-Bold.ttf',
 '/mnt/d/grad23/opensans/fonts/ttf/OpenSans-BoldItalic.ttf',
 '/mnt/d/grad23/opensans/fonts/ttf/OpenSans-CondensedBold.ttf',
 '/mnt/d/grad23/opensans/fonts/ttf/OpenSans-CondensedBoldItalic.ttf',
 '/mnt/d/grad23/opensans/fonts/ttf/OpenSans-CondensedExtraBold.ttf',
 '/mnt/d/grad23/opensans/fonts/ttf/OpenSans-CondensedExtraBoldItalic.ttf',
 '/mnt/d/grad23/opensans/fonts/ttf/OpenSans-CondensedItalic.ttf',
 '/mnt/d/grad23/opensans/fonts/ttf/OpenSans-CondensedLight.ttf',
 '/mnt/d/grad23/opensans/fonts/ttf/OpenSans-CondensedLightItalic.ttf',
 '/mnt/d/grad23/opensans/fonts/ttf/OpenSans-CondensedRegular.ttf',
 '/mnt/d/grad23/opensans/fonts/ttf/OpenSans-CondensedSemiBold.ttf',
 '/mnt/d/grad23/opensans/fonts/ttf/OpenSans-CondensedSemiBoldItalic.ttf',
 '/mnt/d/grad23/opensans/fonts/ttf/OpenSans-ExtraBold.ttf',
 '/mnt/d/grad23/opensans/fonts/ttf/OpenSans-ExtraBoldItalic.ttf',
 '/mnt/d/grad23/opensans/fonts/ttf/OpenSans-Italic.ttf',
 '/mnt/d/gra

### convert

In [36]:
font_processors = [
    font_processing.FontProcessor(faddr)
    for faddr in fonts
]

The following table(s) in the font have been ignored by FontForge
  Ignoring 'DSIG' digital signature table
The glyph named macron is mapped to U+02C9.
But its name indicates it should be mapped to U+00AF.
The following table(s) in the font have been ignored by FontForge
  Ignoring 'DSIG' digital signature table
The glyph named macron is mapped to U+02C9.
But its name indicates it should be mapped to U+00AF.
The following table(s) in the font have been ignored by FontForge
  Ignoring 'DSIG' digital signature table
The glyph named macron is mapped to U+02C9.
But its name indicates it should be mapped to U+00AF.
The following table(s) in the font have been ignored by FontForge
  Ignoring 'DSIG' digital signature table
The glyph named macron is mapped to U+02C9.
But its name indicates it should be mapped to U+00AF.
The following table(s) in the font have been ignored by FontForge
  Ignoring 'DSIG' digital signature table
The glyph named macron is mapped to U+02C9.
But its name indicates i

In [37]:
glyphsets = [
    processor.glyphset_from_unicode_subset(
        font_processing.get_alphabet('all')
    )
    for processor in font_processors
]

In [38]:
jsons = [
    glyphset.to_json()
    for glyphset in glyphsets
]

In [40]:
jsons[5]

{'panose': (0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 'unicoderanges': (-536870145, 1073750043, 40, 0),
 'weight': 800,
 'cap_height': 1462.0,
 'family_name': 'Open Sans Condensed ExtraBold',
 'font_name': 'Open Sans Condensed ExtraBold Italic',
 'glyphs': [{'letter': '!',
   'path': 'M113 516l155 946h375l-246 -946h-284zM147 -25q-159 0 -159 140q0 95 60 169t175 74q68 0 107 -36t39 -101q0 -112 -67 -179t-155 -67z'},
  {'letter': '"',
   'path': 'M616 934l76 528h348l-149 -528h-275zM188 934l76 528h346l-147 -528h-275z'},
  {'letter': '#',
   'path': 'M43 0l123 391h-158l43 234h187l69 209h-170l43 239h199l123 389h241l-125 -389h127l123 389h244l-125 -389h162l-45 -239h-189l-67 -209h172l-43 -234h-201l-125 -391h-239l123 391h-129l-123 -391h-240zM479 625h127l68 209h-127z'},
  {'letter': '$',
   'path': 'M27 170l69 334q73 -66 161 -111t179 -45q59 0 99 34t40 91q0 42 -21.5 70.5t-95.5 68.5q-54 30 -115.5 74.5t-104.5 112t-43 166.5q0 157 62.5 253.5t159.5 143.5t199 53l29 139h105l-29 -139q107 -7 172 -34t117 -62l-146 -283q

In [41]:
target_collection = cli_remote['fonty']['fonts']
target_collection

Collection(Database(MongoClient(host=['ac-5dsaw84-shard-00-00.hquocfa.mongodb.net:27017', 'ac-5dsaw84-shard-00-01.hquocfa.mongodb.net:27017', 'ac-5dsaw84-shard-00-02.hquocfa.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='$external', authmechanism='MONGODB-X509', retrywrites=True, w='majority', replicaset='atlas-32tpky-shard-0', tls=True, tlscertificatekeyfile='/mnt/d/grad23/fonty/auth/X509-cert-6658948524510096383.pem'), 'fonty'), 'fonts')

In [42]:
target_collection.delete_many({'family_name': {'$regex': r'.*Open Sans.*'}})

In [43]:
for font in jsons:
    target_collection.insert_one(font)